In [1]:
# ==============================================================================
# Bloco 1: CONFIGURAÇÃO, CARREGAMENTO E SEGMENTAÇÃO
# ==============================================================================
# Garante que as versões mais recentes das bibliotecas estejam instaladas
!pip install -U splink duckdb --quiet

# --- Importações Essenciais ---
import pandas as pd
import pathlib
import duckdb
import logging
from IPython.display import display, HTML

# --- Importações da API Moderna do Splink v4 ---
# (Não usaremos o Linker ainda, mas já deixamos o ambiente pronto)
from splink import DuckDBAPI, SettingsCreator, Linker
import splink.duckdb.comparison_library as cl


ModuleNotFoundError: No module named 'splink.duckdb'

In [ ]:

# Configuração para manter a saída de logs mais limpa
logging.getLogger("splink").setLevel(logging.WARNING)
print("Bibliotecas importadas com a API correta do Splink v4.")

# --- Caminhos e Conexão com os Dados ---
PASTA_PROCESSED = pathlib.Path("dados_catalogo") / "processed"
ARQUIVO_DE_ENTRADA = PASTA_PROCESSED / 'tabela_dimensao_limpa_para_splink.parquet'

# Conecta ao DuckDB
con = duckdb.connect(database=':memory:')
print(f"\nConectado ao DuckDB (versão: {duckdb.__version__})")


In [ ]:

# --- Segmentação dos Dados ---
# Carrega a tabela dimensão completa em uma view do DuckDB
tabela_completa = "tabela_dimensao_completa"
con.execute(f"CREATE OR REPLACE VIEW {tabela_completa} AS SELECT * FROM read_parquet('{str(ARQUIVO_DE_ENTRADA)}');")
total_registros = con.execute(f"SELECT COUNT(*) FROM {tabela_completa}").fetchone()[0]
print(f"View '{tabela_completa}' criada com {total_registros:,} perfis únicos.")

# Define os critérios para o que consideramos "alta qualidade"
# (Presença de IMDb ID e Diretores, como havíamos discutido)
CRITERIO_ALTA_QUALIDADE = '"IMDb_ID" IS NOT NULL AND "BB_Directors" IS NOT NULL'

# Cria a view para o Universo de Alta Qualidade
tabela_alta_qualidade = "universo_alta_qualidade"
con.execute(f"""
    CREATE OR REPLACE VIEW {tabela_alta_qualidade} AS
    SELECT * FROM {tabela_completa} WHERE {CRITERIO_ALTA_QUALIDADE};
""")

# Cria a view para o Universo de Nicho/Baixa Qualidade
tabela_nicho = "universo_nicho"
con.execute(f"""
    CREATE OR REPLACE VIEW {tabela_nicho} AS
    SELECT * FROM {tabela_completa} WHERE NOT ({CRITERIO_ALTA_QUALIDADE});
""")


# --- Relatório da Segmentação ---
count_alta_qualidade = con.execute(f"SELECT COUNT(*) FROM {tabela_alta_qualidade}").fetchone()[0]
count_nicho = con.execute(f"SELECT COUNT(*) FROM {tabela_nicho}").fetchone()[0]

print("\n" + "="*80)
print(" RELATÓRIO DA SEGMENTAÇÃO DE DADOS")
print("="*80)
print(f"Total de Perfis: {total_registros:,}")
print(f"  - Universo de Alta Qualidade (Filmes/Séries): {count_alta_qualidade:,} ({count_alta_qualidade/total_registros:.1%})")
print(f"  - Universo de Nicho/Dados Esparsos (Outros): {count_nicho:,} ({count_nicho/total_registros:.1%})")
print("\nNosso foco inicial será nos dados de alta qualidade.")

display(HTML("<h4>Amostra do Universo de Alta Qualidade:</h4>"))
display(con.execute(f"SELECT * FROM {tabela_alta_qualidade} LIMIT 5;").fetch_df())

display(HTML("<h4>Amostra do Universo de Nicho/Dados Esparsos:</h4>"))
display(con.execute(f"SELECT * FROM {tabela_nicho} LIMIT 5;").fetch_df())